In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from scipy.stats import spearmanr

In [ ]:
data = pd.read_csv('data_250.csv')

In [ ]:
data.columns

In [ ]:
X = data[['0', '1', '2','3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15','16', '17', '18', '19', '20', '21', '22']].to_numpy()
y = data['ph'].to_numpy()

In [ ]:
SEED = 1

TEST_SIZE = 0.3

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=TEST_SIZE, random_state=SEED)

mtrys = list(range(3, 23, 2))
maxnodes = list(range(60, 200, 10))


In [ ]:
oob_scores = np.zeros((len(mtrys), len(maxnodes)))

print('Training models...')
for i, mtry in enumerate(mtrys):
    print(i)
    for j, maxnode in enumerate(maxnodes):
        print(j)
        model = ensemble.RandomForestRegressor(oob_score=True, n_estimators=500, max_features=mtry, max_leaf_nodes=maxnode, random_state=SEED)
        model.fit(train_X, train_y)
        oob_scores[i, j] = mean_squared_error(model.oob_prediction_, train_y)
print('Finished!')

idx_mtry, idx_maxnode = np.unravel_index(np.argmin(oob_scores), oob_scores.shape)
print(mtrys[idx_mtry], maxnodes[idx_maxnode])


In [ ]:
pd.DataFrame(test_X).to_csv('test_X.csv')
pd.DataFrame(test_y).to_csv('test_y.csv')
pd.DataFrame(train_X).to_csv('train_X.csv')
pd.DataFrame(train_y).to_csv('train_y.csv')

In [ ]:
mtry = 23
maxnode = 200

In [ ]:
tuned_model = ensemble.RandomForestRegressor(max_features=mtry, max_leaf_nodes=maxnode, random_state=SEED, oob_score=True)
tuned_model.fit(train_X, train_y)

In [ ]:
impurity_importances = tuned_model.feature_importances_

mse_importances = permutation_importance(
    tuned_model, test_X, test_y, n_repeats=10, random_state=42, n_jobs=2
).importances_mean

# Normalise
mse_importances = mse_importances / mse_importances.sum()
impurity_importances = impurity_importances / impurity_importances.sum()

correl = round(spearmanr(impurity_importances, mse_importances).correlation, 4)

df = pd.DataFrame.from_dict({
    #'variable': feature_cols,
    '%IncMSE': mse_importances,
    'IncNodePurity': impurity_importances
})
#df = df.set_index('variable')
df = df.sort_index()

df.plot.bar()
plt.xlabel('variable')
plt.ylabel('normalised_importance')
plt.title(f'Spearman correl: {correl}')
plt.xticks(rotation=45)
plt.show()

In [ ]:
yhat = tuned_model.predict(test_X)
mean_squared_error(yhat, test_y)

In [ ]:
X_new = data[['4', '9', '12', '13', '16', '17']].to_numpy()

In [ ]:
SEED = 1

TEST_SIZE = 0.3

train_X, test_X, train_y, test_y = train_test_split(X_new, y, test_size=TEST_SIZE, random_state=SEED)

mtrys = list(range(13, 25, 2))
maxnodes = list(range(20, 90, 10))

oob_scores = np.zeros((len(mtrys), len(maxnodes)))

print('Training models...')
for i, mtry in enumerate(mtrys):
    print(i)
    for j, maxnode in enumerate(maxnodes):
        print(j)
        model = ensemble.RandomForestRegressor(oob_score=True, n_estimators=200, max_features=mtry, max_leaf_nodes=maxnode, random_state=SEED)
        model.fit(train_X, train_y)
        oob_scores[i, j] = mean_squared_error(model.oob_prediction_, train_y)
print('Finished!')

idx_mtry, idx_maxnode = np.unravel_index(np.argmin(oob_scores), oob_scores.shape)
print(mtrys[idx_mtry], maxnodes[idx_maxnode])


In [ ]:
mtry = 6
maxnode = 200

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X_new, y, test_size=TEST_SIZE, random_state=SEED)

tuned_model = ensemble.RandomForestRegressor(max_features=mtry, max_leaf_nodes=maxnode, random_state=SEED, oob_score=True)
tuned_model.fit(train_X, train_y)

In [ ]:
yhat = tuned_model.predict(test_X)
yhat_oob = tuned_model.oob_prediction_

In [ ]:
plt.scatter(test_y, yhat, color='red', marker='.', label='yhat')
#plt.scatter(train_y, yhat_oob, color='teal', marker='.', label='yhat_oob')
plt.axline((4, 4), slope=1, color="black")
plt.xlabel('y')
plt.ylabel('prediction')
plt.legend()

In [ ]:
#plt.scatter(test_y, yhat, color='red', marker='.', label='yhat')
plt.scatter(train_y, yhat_oob, color='teal', marker='.', label='yhat_oob')
plt.axline((4, 4), slope=1, color="black")
plt.xlabel('y')
plt.ylabel('prediction')
plt.legend()

In [ ]:
yhat_all = tuned_model.predict(X_new)

In [ ]:
data['prediction'] = yhat_all

In [ ]:
data['error'] = abs(yhat_all-y)

In [ ]:
pip install plotly

In [ ]:
import plotly.express as px

# Set up mapbox access token
mapbox_access_token = 'pk.eyJ1IjoibWljYXRlbyIsImEiOiJjbGduZjJzeWwwN2ViM2Rwb3JyYnYyYXcxIn0.tsM4wDfrZjYa5ds4s9atSQ'
# Create a scatter map with tooltips
fig = px.scatter_mapbox(data, lat='latitude', lon='longitude', color='error', hover_data=['ph'],
                        zoom=8, center={'lat': data['latitude'].mean(), 'lon': data['longitude'].mean()})
# Set up mapbox style and access token
fig.update_layout(mapbox_style='open-street-map', mapbox_accesstoken=mapbox_access_token)
# Show the map
fig.show()

In [ ]:
import plotly.express as px

# Set up mapbox access token
mapbox_access_token = 'pk.eyJ1IjoibWljYXRlbyIsImEiOiJjbGduZjJzeWwwN2ViM2Rwb3JyYnYyYXcxIn0.tsM4wDfrZjYa5ds4s9atSQ'
# Create a scatter map with tooltips
fig = px.scatter_mapbox(data, lat='latitude', lon='longitude', color='prediction', hover_data=['ph'],
                        zoom=8, center={'lat': data['latitude'].mean(), 'lon': data['longitude'].mean()})
# Set up mapbox style and access token
fig.update_layout(mapbox_style='open-street-map', mapbox_accesstoken=mapbox_access_token)
# Show the map
fig.show()

In [ ]:
import skimage.io as io

im = io.imread('D:/y1/modelling bootcamp/data/s2sr_250mpp.tif')
maps = im[:,:,(4, 9, 12, 13, 16, 17)]

In [ ]:
preds = []
for i in maps:
    preds.append(tuned_model.predict(i))

In [ ]:
import rasterio
import numpy as np

file_name = 'D:/y1/modelling bootcamp/data/s2sr_250mpp.tif'
with rasterio.open(file_name) as src:
    band1 = src.read(1)
    print('Band1 has shape', band1.shape)
    height = band1.shape[0]
    width = band1.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lons= np.array(xs)
    lats = np.array(ys)
    print('lons shape', lons.shape)

In [ ]:
lons_flat = lons.reshape(-1,1).flatten()
lats_flat = lats.reshape(-1,1).flatten()

In [ ]:
preds_flat = [item for sublist in preds for item in sublist]

In [ ]:
preds_flat = np.array(preds_flat)

In [ ]:
fullmap = pd.DataFrame({'longitude': lons_flat, 'latitude': lats_flat, 'predictions': preds_flat})

In [ ]:
fullmap['ph'] = data['ph']

In [ ]:
import plotly.express as px

# Set up mapbox access token
mapbox_access_token = 'pk.eyJ1IjoibWljYXRlbyIsImEiOiJjbGduZjJzeWwwN2ViM2Rwb3JyYnYyYXcxIn0.tsM4wDfrZjYa5ds4s9atSQ'
# Create a scatter map with tooltips
fig = px.scatter_mapbox(fullmap, lat='latitude', lon='longitude', color='predictions', hover_data=['ph'],
                        zoom=8, center={'lat': data['latitude'].mean(), 'lon': data['longitude'].mean()})
# Set up mapbox style and access token
fig.update_layout(mapbox_style='open-street-map', mapbox_accesstoken=mapbox_access_token)
# Show the map
fig.show()